In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

In [2]:
from helpers import Scrapper

## Here I will trying to filter data from FBREF

In [3]:
data = Scrapper()
match = data.get_match('https://fbref.com/en/matches/abf3df21/South-Africa-Italy-August-2-2023-FIFA-Womens-World-Cup')

In [4]:
south = match[0]
south

,South Africa (4-2-3-1),South Africa (4-2-3-1).1
0,1,Kaylin Swart
1,2,Lebohang Ramalepe
2,3,Bongeka Gamede
3,4,Noko Matlou
4,7,Karabo Dhlamini
5,8,Hildah Magaia
6,10,Linda Motlhalo
7,11,Thembi Kgatlana
8,12,Jermaine Seoposenwe
9,13,Bambanani Mbane
